In [7]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml
from pathlib import Path
from utility import *
import pickle

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join

# paths
ctramp_dir = params['ctramp_dir']
model_outputs_dir = params['model_dir']
summary_dir = params['summary_dir']
concept_id = params['concept_id']
preprocess_dir = _join(ctramp_dir, '_pre_process_files')

In [4]:
Path(summary_dir).mkdir(parents=True, exist_ok=True)
Path(preprocess_dir).mkdir(parents=True, exist_ok=True)

In [5]:
purpose = ['Work', 'University', 'School', 'Escort', 'Shopping', 'EatOut', 
           'OthMaint', 'Social', 'OthDiscr', 'WorkBased']

time_period = {1:'EA',2:'AM',3:'MD',4:'PM',5:'EV'} #1 for EA, 2 for AM, 3 for MD, 4 for PM and 5 for EV

In [26]:
%%time


roh_dict = {}

for tripPeriod in time_period:
    
    tp_name = time_period[tripPeriod]
    print(f"reading baseline logsums")
    with open(_join(f'logsums_'+ tp_name +'_BL.pkl'), 'rb') as file:
        ls_bl = pickle.load(file)

    print(f"reading concept logsums")
    with open(_join(preprocess_dir, f'logsums_'+ tp_name +'.pkl'), 'rb') as file:
        ls_cpt = pickle.load(file)

    print(f"reading baseline trips")
    with open(_join(f'trips_'+ tp_name +'_BL.pkl'), 'rb') as file:
        trips_bl = pickle.load(file)

    print(f"reading concept trips")
    with open(_join(preprocess_dir, f'trips_'+ tp_name +'.pkl'), 'rb') as file:
        trips_cpt = pickle.load(file)

    for purp in purpose:
        purp_bl = ls_bl[purp]
        purp_cpt = ls_cpt[purp]
        trp_bl = trips_bl[purp]
        trp_cpt = trips_cpt[purp]

        auto_proportion_diff = purp_cpt['auto_proportion'] - purp_bl['auto_proportion']
        trn_proportion_diff = purp_cpt['trn_proportion'] - purp_bl['trn_proportion']
        nm_proportion_diff = purp_cpt['nm_proportion'] - purp_bl['nm_proportion']
        rh_proportion_diff = purp_cpt['rh_proportion'] - purp_bl['rh_proportion']

        auto_trips = trp_cpt['auto_trips'] - trp_bl['auto_trips']
        trn_trips = trp_cpt['trn_trips'] - trp_bl['trn_trips']
        nm_trips = trp_cpt['nm_trips'] - trp_bl['nm_trips']
        rh_trips = trp_cpt['rh_trips'] - trp_bl['rh_trips']

        auto_trips[auto_trips<0] = 0
        trn_trips[trn_trips<0] = 0
        nm_trips[nm_trips<0] = 0
        rh_trips[rh_trips<0] = 0
        
        print(np.min(auto_trips), np.max(auto_trips))
        print(np.min(trn_trips), np.max(trn_trips))
        print(np.min(nm_trips), np.max(nm_trips))
        print(np.min(rh_trips), np.max(rh_trips))

        auto_benefits = (trp_bl['auto_trips'] + auto_trips) * auto_proportion_diff * 0.5 /0.025
        trn_benefits = (trp_bl['trn_trips'] + trn_trips) * trn_proportion_diff * 0.5 / 0.025
        nm_benefits = (trp_bl['nm_trips'] + nm_trips) * nm_proportion_diff * 0.5 / 0.025
        rh_benefits = (trp_bl['rh_trips'] + rh_trips) * rh_proportion_diff * 0.5 / 0.025

        data_dict = {'auto_benefits' : auto_benefits,
                     'trn_benefits' : trn_benefits,
                     'nm_benefits' : nm_benefits,
                     'rh_benefits' : rh_benefits}

        roh_dict[purp] = data_dict
        
        print(purp, ",", tp_name ,",", np.sum(auto_benefits), ",", np.sum(trn_benefits), np.sum(nm_benefits), np.sum(rh_benefits))
    
    file_path = os.path.join(preprocess_dir, f"benefits_{tp_name}.pkl")
    with open(file_path, "wb") as pickle_file:
        pickle.dump(roh_dict, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
          
    print(f"Data has been written to {file_path} for {purp} and {tripPeriod}.") 
             

reading baseline logsums
reading concept logsums
reading baseline trips
reading concept trips
Work , EA , -166.57955561355132 , 12631.915867988248 -38.56112051553216 -14.905441538430845
University , EA , -0.8924088321306245 , 21.82769145928013 0.07422231537346674 0.03313786329328966
School , EA , -3.1974988594118816 , 36.8806793709783 -0.6752847849079333 -0.1684251680826132
Escort , EA , -155.62005155109026 , 678.1822050149852 -13.003608723776987 -1.3546663230862563
Shopping , EA , -114.36162648295743 , 324.98892314317874 -13.870216345311444 -1.44827095171678
EatOut , EA , -61.83992495332335 , 227.81373332049117 -6.9838445715271575 -0.42711273412065015
OthMaint , EA , -53.536182071182175 , 432.4166200157383 -7.171979293406969 -1.5485895027984782
Social , EA , -21.698343804269925 , 319.86787304494004 -1.538078804097149 -0.22065224467952144
OthDiscr , EA , -40.269022193257655 , 325.2470380001667 -11.548172077729081 -2.6316695782559303
WorkBased , EA , -0.5565704286080053 , 0.064533328623

In [30]:
# non intrazonal 

for tripPeriod in time_period:, 
    
    tp_name = time_period[tripPeriod]
    with open(_join(preprocess_dir, f'benefits_'+ tp_name +'.pkl'), 'rb') as file:
        benefits = pickle.load(file)
        
    for purp in purpose:
        purp_benefit = benefits[purp]
        auto_benefits = purp_benefit['auto_benefits']
        trn_benefits = purp_benefit['trn_benefits']
        nm_benefits = purp_benefit['nm_benefits']
        rh_benefits = purp_benefit['rh_benefits']
        
        np.fill_diagonal(auto_benefits, 0)
        np.fill_diagonal(trn_benefits, 0)
        np.fill_diagonal(nm_benefits, 0)
        np.fill_diagonal(rh_benefits, 0)
        
        print(purp, ",", tp_name ,",", np.sum(auto_benefits), ",", np.sum(trn_benefits), np.sum(nm_benefits), np.sum(rh_benefits))

Work , EA , -167.89434115413877 , 12631.915867988248 -38.47090118712077 -14.911601339720962
University , EA , -0.9072030363730805 , 21.82769145928013 0.0743463597327993 0.03233808232894736
School , EA , -3.3621529761576587 , 36.8806793709783 -0.6582779508050458 -0.17034353098470734
Escort , EA , -158.26483684938535 , 678.1822050149852 -12.902257253152369 -1.3719486987080045
Shopping , EA , -115.16163702971474 , 324.98892314317874 -13.815715710191437 -1.4608387123368693
EatOut , EA , -62.37836708430321 , 227.81373332049117 -6.951609167846131 -0.42508622833540594
OthMaint , EA , -54.29249750401973 , 432.4166200157383 -7.129939082106866 -1.5534391140749146
Social , EA , -21.818035283356394 , 319.86787304494004 -1.5257103854816334 -0.22445154150613683
OthDiscr , EA , -40.709227697198116 , 325.2470380001667 -11.466255572261241 -2.6357529543846474
WorkBased , EA , -0.5688159871303378 , 0.06453332862312644 -0.16674708576917202 -0.021442951697700496
Work , AM , 69438.52362060409 , 165873.55970

In [32]:
%%time
#QAQC
for tripPeriod in time_period:
    
    tp_name = time_period[tripPeriod]
    with open(_join(preprocess_dir, f'benefits_'+ tp_name +'.pkl'), 'rb') as file:
        benefits = pickle.load(file)
        
    for purp in purpose:
        purp_benefit = benefits[purp]
        auto_benefits = purp_benefit['auto_benefits']
        trn_benefits = purp_benefit['trn_benefits']
        nm_benefits = purp_benefit['nm_benefits']
        rh_benefits = purp_benefit['rh_benefits']
        
        np.fill_diagonal(auto_benefits, 0)
        np.fill_diagonal(trn_benefits, 0)
        np.fill_diagonal(nm_benefits, 0)
        np.fill_diagonal(rh_benefits, 0)
        
        print(purp, ",", tp_name ,",", np.max(auto_benefits), np.min(auto_benefits), np.mean(auto_benefits))
        print(purp, ",", tp_name ,",", np.max(trn_benefits), np.min(trn_benefits), np.mean(trn_benefits))
        print(purp, ",", tp_name ,",", np.max(nm_benefits), np.min(nm_benefits), np.mean(nm_benefits))
        print(purp, ",", tp_name ,",", np.max(rh_benefits), np.min(rh_benefits), np.mean(rh_benefits))

Work , EA , 13.38681611272801 -15.26275058509109 -1.5122586353341346e-05
Work , EA , 21.6069622666174 -26.163326756199606 0.001137782472051388
Work , EA , 0.5206214185740743 -0.9223736661111614 -3.465152674556086e-06
Work , EA , 0.2587247074370902 -0.33188148205040624 -1.3431184003962595e-06
University , EA , 0.4973594048653407 -0.44067368886127833 -8.171363110427969e-08
University , EA , 3.0027053401531965 -1.4491879250932038 1.9660647685797125e-06
University , EA , 0.02914672571704302 -0.03212417567505177 6.696528527329236e-09
University , EA , 0.03211139811572228 -0.009090081581816789 2.9127571492835455e-09
School , EA , 0.5543307034123868 -0.43503733422562973 -3.0283598819098394e-07
School , EA , 3.7237573269696944 -1.809196334628862 3.321918146398262e-06
School , EA , 0.2118128142701159 -0.20572027849109942 -5.929244003769387e-08
School , EA , 0.004790693938854229 -0.1783228750406496 -1.5343189885621778e-08
Escort , EA , 3.902725527255275 -1.4942026390272645 -1.4255237225387034e-0

OthDiscr , MD , 18.10391714346361 -24.198954955534138 0.000622763035769911
OthDiscr , MD , 9.454632132913588 -7.55975680301102 -3.756481102314498e-05
OthDiscr , MD , 0.6382186184098415 -0.7678827844477942 -4.166572983296312e-07
WorkBased , MD , 166.61597656737217 -87.09798676889928 0.00016855566062258938
WorkBased , MD , 20.02237817319525 -19.519666682838277 0.0003830631399663409
WorkBased , MD , 26.308209460253078 -22.499128257971964 -5.174630914837871e-05
WorkBased , MD , 1.1279807416257503 -1.040546149942282 -2.4850973360945964e-06
Work , PM , 109.77157548216962 -121.83254018173116 0.0031205925714969866
Work , PM , 171.2653937907762 -122.54499550913604 0.009347650696616463
Work , PM , 10.614763533850313 -17.635823427001633 -1.8504639379761573e-05
Work , PM , 1.3963186609561984 -1.8698503755468243 2.22847415667001e-05
University , PM , 28.544560949180784 -34.11815195980232 6.114431536890476e-05
University , PM , 58.306752361711034 -211.8493897532161 0.00039229591511227065
University 

In [ ]:
for tripPeriod in time_period:
    
    tp_name = time_period[tripPeriod]
    print(f"reading baseline logsums")
    with open(_join(f'logsums_'+ tp_name +'_BL.pkl'), 'rb') as file:
        ls_bl = pickle.load(file)

    print(f"reading concept logsums")
    with open(_join(preprocess_dir, f'logsums_'+ tp_name +'.pkl'), 'rb') as file:
        ls_cpt = pickle.load(file)

    print(f"reading baseline trips")
    with open(_join(f'trips_'+ tp_name +'_BL.pkl'), 'rb') as file:
        trips_bl = pickle.load(file)

    print(f"reading concept trips")
    with open(_join(preprocess_dir, f'trips_'+ tp_name +'.pkl'), 'rb') as file:
        trips_cpt = pickle.load(file)

    for purp in purpose:
        purp_bl = ls_bl[purp]
        purp_cpt = ls_cpt[purp]
        trp_bl = trips_bl[purp]
        trp_cpt = trips_cpt[purp]
        
        for keys in purp_bl:
            